In [1]:
import sys
sys.path.append("../pylinguistics/pylinguistics/")

import Pylinguistics as pl
import pandas as pd
import plotsfunc as pf
import numpy as np
from scipy.stats import spearmanr

%matplotlib inline

In [9]:
def helpInt(x): 
    try:
        return int ( float(x['thumbsup']) * 1000 / (float(x['thumbsup']) + float(x['thumbsdown'])) )
    except:
        return 0

In [10]:
# reset variables
reviews_features = pd.read_csv('reviews.csv.gz', compression='gzip')
reviews_columns = reviews_features.select_dtypes(include=['float64','int']).columns
reviews_features['helpfulness'] = reviews_features.apply(helpInt,axis=1)

reviews_features.shape

(32226, 48)

In [16]:
#keep only the ones that are within +3 to -3 standard deviations

help_mean = reviews_features.helpfulness.mean()
help_std = reviews_features.helpfulness.std()

reviews_features[np.abs(reviews_features.helpfulness-help_mean)<=(2*help_std)]
reviews_features.shape

(32226, 48)

In [17]:
# split class and features
labels = reviews_features["helpfulness"].values
del reviews_features['helpfulness']
features = reviews_features[list(reviews_columns)].values

# clean features
features = np.abs(features)
features[np.isnan(features) == True] = 0

features.shape

(32226, 34)

In [18]:
from sklearn.feature_selection import SelectKBest, f_regression

new_features = SelectKBest(f_regression, k=10).fit_transform(features, labels)
new_features.shape

(32226, 10)

In [ ]:
from sklearn.svm import SVR
from sklearn import cross_validation

estimator = SVR(C=1.0, epsilon=0.2)
scores = cross_validation.cross_val_score(estimator, new_features, labels, cv=10)
scores.mean()